In [ ]:
-----------------------------------------------------------------交叉分析5.0------------------------------------------------

In [3]:
import pandas as pd
import re
import os
import warnings
import numpy as np
from collections import defaultdict
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment, Font, PatternFill, Border, Side
from openpyxl.formatting.rule import DataBarRule
from openpyxl.chart import BarChart, Reference
from scipy.stats import chi2_contingency, fisher_exact  # 新增统计检验库

def extract_subcol_number(subcol, prefix):
    suffix = subcol.split(prefix)[1].strip()
    match = re.search(r'^(\d+)', suffix)
    return int(match.group(1)) if match else 0

def perform_significance_test(observed):
    """执行统计检验并返回p值"""
    try:
        # 尝试卡方检验
        chi2, p, dof, expected = chi2_contingency(observed)
        # 检查期望频数假设（超过20%的单元格期望频数小于5时使用费舍尔精确检验）
        if (expected < 5).sum() / expected.size > 0.2:
            _, p = fisher_exact(observed)
    except:
        # 当出现计算错误时使用费舍尔精确检验
        try:
            _, p = fisher_exact(observed)
        except:
            p = np.nan
    return p



def process_crosstab(
    input_file, 
    output_file, 
    row_questions, 
    col_questions,
    # 新增显著性检验参数
    sig_levels=[0.05, 0.01, 0.001],
    sig_symbols=['*', '**', '***'],
    # 样式配置
    header_height=55,
    header_fill_color="4F81BD",
    header_font_color="FFFFFF",
    header_font_name="微软雅黑",
    header_font_size=12,
    # 数据条配置
    freq_data_bar_color="638EC6",
    percent_data_bar_color="C00000",
    data_bar_min_length=15,
    data_bar_max_length=100,
    # 格式配置
    percent_format="0.00%",
    data_column_width=20,  # C列及之后的固定宽度（None表示自动调整）
    max_column_width=40 #【可调整最大列宽】
):

    
    # === 数据准备 ===
    try:
        df = pd.read_excel(input_file)
        df.columns = [str(col).strip() for col in df.columns]  # 统一清理列名

        
    
        # ===== 新增：变量重新编码功能 =====
        def recode_variable(df, source_col, new_col, rules):
            """
            重新编码变量
            参数：
            - df: 原始DataFrame
            - source_col: 原始列名
            - new_col: 新列名
            - rules: 编码规则字典，格式：
                {
                    'method': 'map'/'cut',  # 映射方式
                    'config': {
                        # map方式示例：
                        'mapping': {1:1, 2:2, 3:3, 4:4, 5:4}, 
                        # cut方式示例：
                        'bins': [0, 30, 50, 120],
                        'labels': ['青年','中年','老年']
                    }
                }
            """
            if source_col not in df.columns:
                warnings.warn(f"原始列 {source_col} 不存在，跳过重新编码")
                return df
            
            if rules['method'] == 'map':
                df[new_col] = df[source_col].map(rules['config']['mapping'])
            elif rules['method'] == 'cut':
                df[new_col] = pd.cut(
                    df[source_col], 
                    bins=rules['config']['bins'],
                    labels=rules['config']['labels'],
                    ordered=False
                )
            else:
                raise ValueError("不支持的编码方式")
            
            return df
        
        # ===== 编码规则配置 =====
        recode_rules = [
            # 示例：Q21重新编码
            {
                'source_col': 'Q47.请问您的年龄是？',
                'new_col': 'recode_请问您的年龄是？',
                'rules': {
                    'method': 'map',
                    'config': {
                        'mapping': {
                            1: "1：9岁及以下",
                            2: "2：10-12岁",
                            3: "3：13-14岁",
                            4: "4：15-17岁",
                            5: "5：18-19岁",
                            6: "6：20-24岁",
                            **{k:"7：25岁及以上" for k in range(7,13)},  #  range(start, end) 的区间是 左闭右开 的。例如，range(4,9) → 生成 4,5,6,7,8
                            13: "8：不愿意透露",
                            # **{k:5 for k in range(9,13)},  
                            np.nan: "缺失"  # 显式保留缺失
                        }
                    }
                }
            },
            {
                'source_col': 'Q48.请问您的职业是？',
                'new_col': 'recode_请问您的职业是？',
                'rules': {
                    'method': 'map',
                    'config': {
                        'mapping': {
                            1: "1：小学生",
                            2: "2：初中生",
                            3: "3：高中生",
                            4: "4：大学生（包括专科、本科、研究生等）",
                            **{k:"5：工作人群" for k in range(5,14)},  #  range(start, end) 的区间是 左闭右开 的。例如，range(4,9) → 生成 4,5,6,7,8
                            14: "6：其他",
                            15: "7：不愿意透露",
                            np.nan: "缺失"  # 显式保留缺失
                        }
                    }
                }
            },
             {
                'source_col': 'Q1.就目前的体验而言，您对这款游戏的整体满意度评价如何？',
                'new_col': 'Q1_满意度分人群',
                'rules': {
                    'method': 'map',
                    'config': {
                        'mapping': {
                            **{k:"不满意" for k in range(1,4)}, 
                            **{k:"满意" for k in range(4,6)}, 
                            np.nan: "缺失"  # 显式保留缺失
                        }
                    }
                }
            },

        ]
        
        # 应用所有编码规则
        for rule in recode_rules:
            df = recode_variable(df, **rule)
    


    
    except Exception as e:
        raise Exception(f"输入文件读取失败: {str(e)}")

    # === 识别用户配置的多选题根 ===
    user_multi_roots = set()
    for q in row_questions + col_questions:
        q_clean = str(q).strip()
        if re.fullmatch(r'^Q\d+\.$', q_clean):  # 严格匹配 Q数字. 格式
            user_multi_roots.add(q_clean)
    
    # === 构建多选题字典 ===
    multi_choice_dict = defaultdict(list)
    for col in df.columns:
        col_clean = str(col).strip()
        for root in user_multi_roots:
            if col_clean.startswith(root):
                multi_choice_dict[root].append(col)
    
    # 排序并过滤有效多选题
    multi_choice_dict = {
        k: sorted(v, key=lambda x: extract_subcol_number(x, k))
        for k, v in multi_choice_dict.items() if len(v) > 1
    }







    
    # === 验证问题有效性 ===
    def validate_questions(questions):
        valid = []
        invalid = []
        for q in questions:
            q_clean = str(q).strip()

            if q_clean in df.columns:
                valid.append(('single', q_clean))

            elif q_clean in multi_choice_dict:
                valid.append(('multi', q_clean))

            else:
                invalid.append(q)
        return valid, invalid
    
    valid_rows, invalid_rows = validate_questions(row_questions)
    valid_cols, invalid_cols = validate_questions(col_questions)
    
    if invalid_rows or invalid_cols:
        warnings.warn(f"无效问题将被跳过：行问题={invalid_rows}，列问题={invalid_cols}")

    # === 问题有效性验证 ===
    invalid_questions = []
    valid_rows = []

    # 处理行问题
    for q in row_questions:
        found = False
        if re.match(r'^Q\d+\.', str(q)):
            root = re.match(r'^(Q\d+\.)', str(q)).group(1)
            if root in multi_choice_dict:
                valid_rows.append(('multi', root))
                found = True
        if not found and q in df.columns:
            valid_rows.append(('single', q))
            found = True
        if not found:
            invalid_questions.append(q)


        # === 列问题处理 ===
    # === 修改后的列条件生成代码 ===
    col_conditions = []
    col_totals = {}
    seen_cols = defaultdict(int)  # 记录列问题出现次数
    
    for q in col_questions:  # 保留原始顺序，不跳过重复项
        q_clean = str(q).strip()
        seen_cols[q_clean] += 1
        instance_id = seen_cols[q_clean]
    
        # === 处理多选题 ===
        if re.match(r'^Q\d+\.', q_clean):
            root = re.match(r'^(Q\d+\.)', q_clean).group(1)
            if root in multi_choice_dict:
                subcols = sorted(multi_choice_dict[root], key=lambda x: extract_subcol_number(x, root))
                example_subcol = subcols[0]
                rest_part = example_subcol.split(root)[1].strip()
                if ':' in rest_part:
                    question_text, _ = rest_part.split(':', 1)
                    question_text = question_text.strip()
                else:
                    question_text = rest_part
                full_question = f"{root}{question_text} #{instance_id}"  # 唯一标识
    
                for subcol in subcols:
                    rest_subcol = subcol.split(root)[1].strip()
                    if ':' in rest_subcol:
                        _, option_text = rest_subcol.split(':', 1)
                        option_text = option_text.strip()
                    else:
                        option_text = rest_subcol
                    label = f"{full_question}\n{option_text}"
                    cond = df[subcol] == 1
                    col_conditions.append((label, cond))
                    col_totals[label] = cond.sum()
    
                total_label = f"{full_question}\n总计"
                total_cond = (df[subcols] == 1).any(axis=1)
                col_conditions.append((total_label, total_cond))
                col_totals[total_label] = total_cond.sum()
                continue
    
        # === 处理单选题 ===
        if q_clean in df.columns:
            values = df[q_clean].dropna().unique()
            try:
                sorted_values = sorted(values, key=lambda x: int(re.match(r'^(\d+)', str(x)).group(1)))
            except:
                sorted_values = values
            unique_question = f"{q_clean} #{instance_id}"  # 唯一标识
            for value in sorted_values:
                label = f"{unique_question}\n{value}"
                cond = df[q_clean] == value
                col_conditions.append((label, cond))
                col_totals[label] = cond.sum()
            total_label = f"{unique_question}\n总计"
            total_cond = df[q_clean].notna()
            col_conditions.append((total_label, total_cond))
            col_totals[total_label] = total_cond.sum()
        else:
            warnings.warn(f"无效问题被跳过：{q}")



    # === 行维度条件生成 ===
    row_conditions = []
    for q_type, q in valid_rows:
        if q_type == 'multi':
            root = re.match(r'^(Q\d+\.)', q).group(1)
            subcols = sorted(multi_choice_dict[root],
                           key=lambda x: extract_subcol_number(x, root))
            for subcol in subcols:
                option = subcol.split(root)[1].strip()
                cond = df[subcol] == 1
                row_conditions.append(((q, option), cond))  # 元组格式
            total_cond = (df[subcols] == 1).any(axis=1)
            row_conditions.append(((q, '总计'), total_cond))
        else:





            
            # 处理单选题
            values = df[q].dropna().unique()
            try:
                sorted_values = sorted(values, key=lambda x: int(re.match(r'^(\d+)', str(x)).group(1)))
            except:
                sorted_values = values
            for value in sorted_values:
                cond = df[q] == value
                row_conditions.append(((q, str(value)), cond))  # 元组形式 (问题, 值)
            # 生成总计行
            total_cond = df[q].notna()
            row_conditions.append(((q, '总计'), total_cond))




    # === 交叉统计计算 ===
    freq_results = []
    for (r_question, r_option), r_cond in row_conditions:
        row_data = {}
        for c_label, c_cond in col_conditions:
            count = (r_cond & c_cond).sum()
            row_data[c_label] = count
        freq_results.append(row_data)

    # === 创建多级索引 ===
    index = pd.MultiIndex.from_tuples(
        [(rl[0], rl[1]) for rl, _ in row_conditions],  # 提取问题和选项
        names=['问题', '选项']
    )

    freq_df = pd.DataFrame(
        freq_results,
        index=index,
        columns=[cl for cl, _ in col_conditions]
    )





    
    # === 百分比计算 ===
    percent_df = freq_df.copy()
    for col in percent_df.columns:
        total = col_totals[col]
        percent_df[col] = (freq_df[col] / total).round(3)

    # === 百分比计算 ===
    percent_df = freq_df.copy()
    for col in percent_df.columns:
        total = col_totals[col]
        percent_df[col] = (freq_df[col] / total).round(3)

    # === 构建最终表格 ===
    freq_df = freq_df.add_suffix("（频数）")
    percent_df = percent_df.add_suffix("（百分比）")    # 【可调整是否显示百分比】

    columns_order = []
    for orig_col in [cl for cl, _ in col_conditions]:
        columns_order.append(f"{orig_col}（频数）")
        columns_order.append(f"{orig_col}（百分比）")   # 【可调整是否显示百分比】

    combined_df = pd.concat([freq_df, percent_df], axis=1)[columns_order]

    # === 新增：显著性检验计算 ===
    sig_results = []
    for r_label, r_cond in row_conditions:
        row_sig = {}
        for c_label, c_cond in col_conditions:
            # 构建列联表
            a = (r_cond & c_cond).sum()
            b = (r_cond & ~c_cond).sum()
            c = (~r_cond & c_cond).sum()
            d = (~r_cond & ~c_cond).sum()
            observed = np.array([[a, b], [c, d]])
            
            # 执行检验
            p_value = perform_significance_test(observed)
            row_sig[c_label] = p_value
        sig_results.append(row_sig)
    
    sig_df = pd.DataFrame(sig_results, 
                        index=[rl for rl, _ in row_conditions],
                        columns=[cl for cl, _ in col_conditions])

    # === 新增：生成带星号标记的显著性结果 ===
    formatted_sig_df = sig_df.copy()
    for col in formatted_sig_df.columns:
        formatted_sig_df[col] = formatted_sig_df[col].apply(
            lambda p: ''.join([s for l, s in zip(sig_levels, sig_symbols) if p <= l]) + 
                f"({p:.3f})" if not pd.isna(p) else ""
        )
    
    # === 处理已有文件 ===
    if os.path.exists(output_file):
        try:
            os.remove(output_file)
        except PermissionError:
            raise PermissionError(f"请关闭正在使用的文件：{output_file}")

    # =========================================================== Excel输出 ============================================================
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # === 提前定义样式 ===
        header_fill = PatternFill(
            start_color=header_fill_color,
            end_color=header_fill_color,
            fill_type="solid"
        )
        header_font = Font(
            name=header_font_name,
            size=header_font_size,
            bold=True,
            color=header_font_color
        )
        thin_border = Border(
            left=Side(style='thin'),
            right=Side(style='thin'),
            top=Side(style='thin'),
            bottom=Side(style='thin')
        )
    

        # === 交叉分析sheet ===
        combined_df.to_excel(writer, sheet_name='交叉分析', merge_cells=True)  # 确保 merge_cells=True
        worksheet = writer.sheets['交叉分析']


        
        # === 设置百分比格式 ===
        for col_idx in range(1, worksheet.max_column + 1):
            cell_value = worksheet.cell(row=1, column=col_idx).value
            if cell_value and "百分比" in cell_value:
                col_letter = get_column_letter(col_idx)
                for row in worksheet.iter_rows(
                    min_row=2, 
                    max_row=worksheet.max_row,
                    min_col=col_idx,
                    max_col=col_idx
                ):
                    for cell in row:
                        cell.number_format = percent_format
       
        # 标题样式
        worksheet.row_dimensions[1].height = header_height

        # 设置其他行行高（例如设置为20）
        for row_idx in range(2, worksheet.max_row + 1):
            worksheet.row_dimensions[row_idx].height = 20  # 设置普通行高度
        for cell in worksheet[1]:
            cell.font = Font(
                name=header_font_name,
                size=header_font_size,
                bold=True,
                color=header_font_color
            )

        # === 数据条设置===
        freq_rule = DataBarRule(
            start_type='num', 
            start_value=0,
            end_type='max', 
            color=freq_data_bar_color,
            showValue="None",
            minLength=data_bar_min_length,
            maxLength=data_bar_max_length
        )
        percent_rule = DataBarRule(
            start_type='num', 
            start_value=0,
            end_type='max', 
            color=percent_data_bar_color,
            showValue="None",
            minLength=data_bar_min_length,
            maxLength=data_bar_max_length
        )

        # 预先生成有效行列表
        valid_rows = []
        for row_idx in range(2, worksheet.max_row + 1):
            row_label = worksheet.cell(row=row_idx, column=1).value or ""
            # 清理标签中的换行符和空格
            clean_label = row_label.replace('\n', '').replace(' ', '')
            # 判断是否为总计行（支持多种格式）
            if any(keyword in clean_label for keyword in ["总计", "Total", "合计"]):
                continue
            valid_rows.append(row_idx)

        # 应用数据条到所有有效列
        for col_idx in range(1, worksheet.max_column + 1):
            header_cell = worksheet.cell(row=1, column=col_idx)
            header_value = header_cell.value or ""
            
            # 确定规则类型
            if "频数" in header_value:
                rule = freq_rule
            elif "百分比" in header_value:
                rule = percent_rule
            else:
                continue

            # 仅当存在有效数据行时应用
            if valid_rows:
                col_letter = get_column_letter(col_idx)
                data_range = f"{col_letter}{min(valid_rows)}:{col_letter}{max(valid_rows)}"
                worksheet.conditional_formatting.add(data_range, rule)



        # === 格式优化 ===
        # 设置列宽
        # 设置A列宽度
        worksheet.column_dimensions['A'].width = 25  # 问题列宽
        # 设置B列宽度
        worksheet.column_dimensions['B'].width = 25  # 选项列宽
        
        # 设置C列及之后的宽度
        for col in worksheet.columns:
            col_letter = get_column_letter(col[0].column)
            
            # 跳过已设置的A、B列
            if col_letter in ['A', 'B']: 
                continue
                
            # 计算最大列宽
            max_length = 0
            for cell in col:
                try:
                    # 处理换行文本：取最长行的长度
                    if cell.value and '\n' in str(cell.value):
                        line_lengths = [len(line) for line in str(cell.value).split('\n')]
                        cell_length = max(line_lengths)
                    else:
                        cell_length = len(str(cell.value))
                    max_length = max(max_length, cell_length)
                except:
                    pass
            
            # 设置列宽（使用自定义宽度或自动调整）
            if data_column_width:  # 如果设置了固定宽度
                worksheet.column_dimensions[col_letter].width = data_column_width
            else:  # 否则自动调整宽度
                adjusted_width = min(max_length + 2, max_column_width)
                worksheet.column_dimensions[col_letter].width = adjusted_width

        # 设置边框
        thin_border = Border(
            left=Side(style='thin'),
            right=Side(style='thin'),
            top=Side(style='thin'),
            bottom=Side(style='thin')
        )
        for row in worksheet.iter_rows():
            for cell in row:
                cell.border = thin_border
                cell.alignment = Alignment(
                    wrap_text=True, 
                    vertical='top',
                    horizontal='left'
                )

        # === 设置全局字体 ===
        for row in worksheet.iter_rows():
            for cell in row:
                cell.font = Font(name="微软雅黑")  # 保留原有其他属性

        # === 冻结前两列和第一行 ===
        worksheet.freeze_panes = "C2"
        worksheet.sheet_view.showGridLines = False   # 隐藏网格线

        # 新增显著性检验sheet
        sig_df.to_excel(writer, sheet_name='显著性检验')
        formatted_sig_df.to_excel(writer, sheet_name='带星号显著性')

        # 设置显著性sheet样式（复用已定义的样式变量）
        for sheet_name in ['显著性检验', '带星号显著性']:
            sheet = writer.sheets[sheet_name]
            for cell in sheet[1]:  # 设置标题行样式
                cell.fill = header_fill
                cell.font = header_font
                cell.border = thin_border
            # 设置数字格式
            for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):
                for cell in row[1:]:
                    if sheet_name == '显著性检验':
                        cell.number_format = '0.000'
                    cell.alignment = Alignment(horizontal='center')


    return combined_df, sig_df



if __name__ == "__main__":
    
    # 【可调整行列变量】
    ROW_QUESTIONS = [
        "Q45.",
        "Q1.就目前的体验而言，您对这款游戏的整体满意度评价如何？", 
        "Q1_满意度分人群",
        "Q2.您在多大程度上愿意把MC移动版推荐给其他人呢？", 
        "Q3.您对MC移动版当前版本在以下方面的体验感受如何？*1星表示非常不满意，5星表示非常满意，星值越高表示满意度越高:社交体验（氛围是否热闹、玩家是否友好）", 
        "Q3.您对MC移动版当前版本在以下方面的体验感受如何？*1星表示非常不满意，5星表示非常满意，星值越高表示满意度越高:玩法乐趣（游戏玩法是否有趣、是否好玩）", 
        "Q3.您对MC移动版当前版本在以下方面的体验感受如何？*1星表示非常不满意，5星表示非常满意，星值越高表示满意度越高:基础性能（运行流畅性等）", 
        "Q3.您对MC移动版当前版本在以下方面的体验感受如何？*1星表示非常不满意，5星表示非常满意，星值越高表示满意度越高:UGC玩法（模组、地图等）", 
        "Q3.您对MC移动版当前版本在以下方面的体验感受如何？*1星表示非常不满意，5星表示非常满意，星值越高表示满意度越高:付费体验（购买皮肤、模组）",
        "Q4.就目前的体验而言，您对MC手机版基础性能相关问题的满意度评价如何？*1星表示非常不满意，5星表示非常满意，星值越高表示满意度越高:BUG",
        "Q4.就目前的体验而言，您对MC手机版基础性能相关问题的满意度评价如何？*1星表示非常不满意，5星表示非常满意，星值越高表示满意度越高:卡顿/延迟",
        "Q4.就目前的体验而言，您对MC手机版基础性能相关问题的满意度评价如何？*1星表示非常不满意，5星表示非常满意，星值越高表示满意度越高:发热",
        "Q4.就目前的体验而言，您对MC手机版基础性能相关问题的满意度评价如何？*1星表示非常不满意，5星表示非常满意，星值越高表示满意度越高:闪退",
        "Q46.请问您的性别是？",
        "recode_请问您的年龄是？",
        "recode_请问您的职业是？"
    ]
    
    COL_QUESTIONS = [
        "Q47.请问您的年龄是？",
        "Q1.就目前的体验而言，您对这款游戏的整体满意度评价如何？", 
        "Q1_满意度分人群",
        "Q2.您在多大程度上愿意把MC移动版推荐给其他人呢？", 
        "Q3.您对MC移动版当前版本在以下方面的体验感受如何？*1星表示非常不满意，5星表示非常满意，星值越高表示满意度越高:社交体验（氛围是否热闹、玩家是否友好）", 
        "Q3.您对MC移动版当前版本在以下方面的体验感受如何？*1星表示非常不满意，5星表示非常满意，星值越高表示满意度越高:玩法乐趣（游戏玩法是否有趣、是否好玩）", 
        "Q3.您对MC移动版当前版本在以下方面的体验感受如何？*1星表示非常不满意，5星表示非常满意，星值越高表示满意度越高:基础性能（运行流畅性等）", 
        "Q3.您对MC移动版当前版本在以下方面的体验感受如何？*1星表示非常不满意，5星表示非常满意，星值越高表示满意度越高:UGC玩法（模组、地图等）", 
        "Q3.您对MC移动版当前版本在以下方面的体验感受如何？*1星表示非常不满意，5星表示非常满意，星值越高表示满意度越高:付费体验（购买皮肤、模组）",
        "Q4.就目前的体验而言，您对MC手机版基础性能相关问题的满意度评价如何？*1星表示非常不满意，5星表示非常满意，星值越高表示满意度越高:BUG",
        "Q4.就目前的体验而言，您对MC手机版基础性能相关问题的满意度评价如何？*1星表示非常不满意，5星表示非常满意，星值越高表示满意度越高:卡顿/延迟",
        "Q4.就目前的体验而言，您对MC手机版基础性能相关问题的满意度评价如何？*1星表示非常不满意，5星表示非常满意，星值越高表示满意度越高:发热",
        "Q4.就目前的体验而言，您对MC手机版基础性能相关问题的满意度评价如何？*1星表示非常不满意，5星表示非常满意，星值越高表示满意度越高:闪退",
        "Q46.请问您的性别是？",
        "recode_请问您的年龄是？",
        "recode_请问您的职业是？"
    ]
    
    # 【可调整文件路径】
    INPUT_FILE = r"C:\Users\lijinghui03\Desktop\UR工作资料-2025\【116-G79】【25年7月满意度&官网】新界面调研-202507\survey_83426_w_0【合并】.xlsx"
    OUTPUT_FILE = r"C:\Users\lijinghui03\Desktop\UR工作资料-2025\【116-G79】【25年7月满意度&官网】新界面调研-202507\survey_83426_w_0【合并】[corsstable].xlsx"

    # 执行分析
    crosstab_df, sig_df = process_crosstab(
        input_file=INPUT_FILE,
        output_file=OUTPUT_FILE,
        row_questions=ROW_QUESTIONS,
        col_questions=COL_QUESTIONS,
        # 优化数据条颜色
        freq_data_bar_color="9BC2E6",  # 更浅的蓝色
        percent_data_bar_color="F4B084",  # 更浅的橙色
    )
    print(f"最终分析完成！交叉表保存在【交叉分析】sheet，检验结果在【显著性检验】和【带星号显著性】sheet！文件已保存至：{OUTPUT_FILE}")


最终分析完成！交叉表保存在【交叉分析】sheet，检验结果在【显著性检验】和【带星号显著性】sheet！文件已保存至：C:\Users\lijinghui03\Desktop\UR工作资料-2025\【116-G79】【25年7月满意度&官网】新界面调研-202507\survey_83426_w_0【合并】[corsstable].xlsx


In [ ]:
-----------------------------------------------------------------路径检验-----------------------------------------------------------------

In [3]:

# 问题1：
# 修正路径，确保文件路径存在

import os

INPUT_FILE = r"C:\Users\wb.lijinghui01\Desktop\UR工作资料-2025\【0-常驻问卷】1、2、3、7日\【首日】0101-0515\survey_75176_w_0.xlsx"

if os.path.exists(INPUT_FILE):
    print("文件存在，可继续执行")
else:
    print("文件不存在！请检查路径：", INPUT_FILE)

#几个修正方式：1.手动重新输入路径：避免复制，直接手打路径字符串。2.加个r。

# 问题2：在正确的Python环境中安装各种包？
import sys
print(sys.executable)

# 在Windows命令提示符（cmd）中运行，且替换为上面打印的实际路径
"C:\Path\To\Your\Python.exe" -m pip install ipfn



文件存在，可继续执行


In [ ]:
-----------------------------------------------------------------文本题分析2.0-----------------------------------------------------------------

In [9]:
import os
os.environ["LOKY_PICKLER"] = "pickle"
os.environ["JOBLIB_START_METHOD"] = "loky"  # 新增在文件最开头
import pandas as pd
import jieba
import re
import os
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="joblib")

# ================== 功能模块 ==================
# 1. 变量识别模块
def load_data(file_path, text_var, other_vars):
    df = pd.read_excel(file_path)
    return df[[text_var] + other_vars].copy()

# 2. 文本清洗模块
def clean_text(df, text_var, invalid_words=['无', ' ']):
    df[text_var] = df[text_var].str.strip()
    cond = df[text_var].isin(invalid_words) | df[text_var].isna()
    return df[~cond].reset_index(drop=True)

# 3. 标签匹配模块（核心修改） # (修改)
def manual_tagging(text, tag_keywords, 
                  negation_words={"不", "没", "未", "无", "非", "勿"},  # (新增)否定词表
                  max_context=3):  # (新增)上下文检测范围
    """
    增强版标签匹配逻辑：
    1. 分句处理避免跨句误判
    2. 整句级否定检测
    3. 关键词上下文否定检测
    """
    matched_tags = set()
    matched_keywords = set()
    
    # (新增) 分句处理（支持中文标点）
    sentences = re.split(r'[,.，。！？；\n]', text)
    
    for sent in sentences:
        sent = sent.strip()
        if not sent:
            continue
        
        # (新增) 整句否定检测
        if any(nw in sent for nw in negation_words):
            continue
            
        # 遍历所有标签和关键词
        for tag, keywords in tag_keywords.items():
            for kw in keywords:
                kw_pos = sent.find(kw)
                if kw_pos == -1:
                    continue
                
                # (新增) 上下文否定检测
                context_start = max(0, kw_pos - max_context)
                context_end = min(len(sent), kw_pos + len(kw) + max_context)
                context = sent[context_start:context_end]
                
                if not any(nw in context for nw in negation_words):
                    matched_tags.add(tag)
                    matched_keywords.add(kw)
    
    return ", ".join(matched_tags), ", ".join(matched_keywords)



    
# 4. 词云分析模块
def generate_wordcloud(texts, stopwords, save_path=None, font_path='simhei.ttf'):
    # 新增文本预处理
    def preprocess(text):
        # 移除标点符号（新增）
        text = re.sub(r'[^\w\s]', '', text)
        # 精确分词（修改）
        return [word for word in jieba.lcut(text) 
                if len(word) > 1 and word not in stopwords]
    
    # 合并所有文本并预处理
    all_words = []
    for text in texts:
        all_words.extend(preprocess(text))
    
    # 手动统计词频（核心修改）
    word_freq = Counter(all_words)
    
    # 过滤低频词（新增）
    min_freq = 2  # 可调节参数
    filtered_freq = {k:v for k,v in word_freq.items() if v >= min_freq}
    
    # 生成词云
    wc = WordCloud(
        font_path=font_path,
        background_color='white',
        width=1600,
        height=1200,
        max_words=40,
        colormap='viridis'  # 新增统一配色
    )
    wc.generate_from_frequencies(filtered_freq)  # 关键修改
    
    # 绘图设置
    plt.figure(figsize=(16, 12))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close()

# 5. 文本聚类模块
def text_clustering(texts, n_clusters=10, max_samples=20):
    tfidf = TfidfVectorizer(max_features=500)
    X = tfidf.fit_transform(texts)
    
    kmeans = KMeans(
        n_clusters=n_clusters,
        n_init=10,
        random_state=42,
        init='k-means++'
    )
    
    clusters = kmeans.fit_predict(X)
    
    results = []
    for cluster_id in range(n_clusters):
        cluster_texts = [t for t, c in zip(texts, clusters) if c == cluster_id]
        results.append({
            "cluster": cluster_id,
            "count": len(cluster_texts),
            "examples": cluster_texts[:max_samples]
        })
    return pd.DataFrame(results), clusters

# 6. 结果输出模块
def export_results(df, cluster_df, output_path):
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with pd.ExcelWriter(output_path) as writer:
        df.to_excel(writer, sheet_name='分析结果', index=False)
        cluster_df.to_excel(writer, sheet_name='聚类统计', index=False)

# ================== 主流程 ==================
if __name__ == "__main__":
    # ========== 参数配置 ==========
    import multiprocessing
    multiprocessing.freeze_support()



    # 文件路径
    FILE_PATH = r"C:\Users\lijinghui03\Desktop\UR工作资料-2025\【116-G79】【25年7月满意度&官网】新界面调研-202507\survey_83426_w_0【合并】.xlsx"
    OUTPUT_PATH = r"C:\Users\lijinghui03\Desktop\UR工作资料-2025\【116-G79】【25年7月满意度&官网】新界面调研-202507\survey_83426_w_0【合并】[Q37].xlsx"

    WORDCLOUD_PATH = os.path.splitext(OUTPUT_PATH)[0] + "_词云图.png"  # 自动生成路径新增词云图路径配置

    
    TEXT_VAR = "Q37.（非必填）您希望《我的世界：中国版》官网能新增什么功能或内容？"
    OTHER_VARS = ["Y1.uiduid", "结束答题时间"]
    STOPWORDS = ["的", "了", "是", " "
                 , '有些'
                 , '因为', '游戏', '世界', '而且', '非常', '建议', '希望', '什么', '不要', '一点', '司马', '必须', '他妈的', '网易', '进入', '无法'
                 , '一个', '觉得', '其他', '感觉', '知道', '没什么', '哪些', '或者', '不能', '增加', '那个', '还有', '好玩','框上','除了'
                 , '这个', '没有', '不知道', '愿意', '玩家', '满意', '意见', '时候', '有点', '那些', '死妈', '随便', '卡顿', '闪退'
                 , '比较', '一些', '很多', '可以', '只有', '一下', '东西', '取消', '自己', '这样', '问题', '直接', '更好', '我', '生存', '迷你', '手游', '原版', '钻石'
                 , '看到', '现在', '但是', '所以', '还是', '多一点', '不用', '我们', '里面', '并且', '我们', '里面','第一人称','第三人称'
                 , '就是', '继续',  '手机', '旧官', '新官', '官网', '华为', '不会', '那么', '界面','更新'
                ]
    invalid_words = ['无', ' ', '没有', '不知道', '愿意',r'[，。！？；\n]', r'\d+', r'[【】（）《》“”‘’]'] # 清洗整段文本
    NEGATION_WORDS = {
        # 基础否定
        "不", "没", "未", "无", "非", "勿", "无需", "别", "莫",
        # 语义否定
        "不再", "不会", "不需要", "不应该", "不存在", "未遇到",
        # 转折关联词
        "但是", "然而", "其实", "实际上", "尽管",
        # 方言/口语变体
        "冇", "唔", "冇得", "罢了"
    }


    TAG_KEYWORDS = {
    # 核心玩法（出现频率38.7%）
    "核心玩法": [
        r"(?:非常|超级|特别|极其|十分|好)好玩", "生存", "创造", "红石", "指令", "合成表", "附魔", "村民交易",
        "下界传送门", "末地船", "刷怪塔", "生电", "挖矿", "建筑",
        "挖矿", "建筑",
        "冒险", "探索", "起床战争", "空岛生存", "PVP", "跑酷"
    ],
    
    # 社交与联机（出现频率29.3%）
    "社交联机": [
        "设计", "联机","好友", "服务器", "租赁服", "联机大厅", "多人游戏", "社交",
        "组队", "语音", "聊天", "领地", "经济系统", "小游戏",
        "RPG", "起床战争", "玩家素质", "举报系统"
    ],
    
    # 性能与优化（出现频率22.1%）
    "性能体验": [
        "卡顿", "闪退", "加载慢", "掉帧", "内存不足", "FPS",
        "渲染错误", "崩溃报告", "发热", "网络延迟", "掉线",
        "按键冲突", "触控优化", "横屏适配", "竖屏操作"
    ],
    
    # 内容更新（出现频率18.6%）
    "版本特性": [
        "1.20", "考古系统", "樱花木", "骆驼", "悦灵", "幽匿系列",
        "红树林", "Trails & Tales", "试炼大厅", "紫水晶",
        "考古", "风铃", "惊变100天", "魔法指令"
    ],
    
    # 模组生态（出现频率15.9%）
    "模组组件": [
        "模组","组件","MOD","mod","光影", "材质包", "数据包", "JEI", "Optifine", 
        "连锁挖矿", "惊变", "魔法", "枪械", "机甲",
        "家具", "科技模组", "神奇宝贝", "暮色森林",
        "等价交换", "工业时代", "匠魂"
    ],
    
    # 商业化（出现频率12.4%）
    "商业化": [
        "钻石", "绿宝石", "皮肤", "会员", "抽奖", "充值",
        "付费模组", "免费试玩", "福利活动", "礼包",
        "4D皮肤", "传奇皮肤", "坐骑", "特效"
    ],
    
    # UI/UX（出现频率9.8%）
    "界面交互": [
        "UI", "HUD", "快捷栏", "合成界面", "地图标记", "进度系统",
        "配方书", "云存档", "自定义按键", "摇杆操作",
        "触控反馈", "字幕", "物品栏", "成就系统"
    ],
    
    # 技术问题（出现频率8.2%）
    "技术异常": [
        "解压失败", "模组冲突", "存档丢失", "物品复制",
        "区块加载", "生物生成", "材质错误", "指令失效",
        "交易BUG", "传送异常", "刷怪笼失效", "红石故障"
    ],
    
    # 情怀与社区（出现频率7.5%）
    "情怀文化": [
        "童年回忆", "十年老玩家", "此生无悔入MC", "方块人", "热爱mc", "喜欢mc",
        "信仰", "游戏主播", "MC社区", "玩家创作", "热爱", "经典",
        "建筑大赛", "红石发明", "速通记录", "模组作者"
    ],
    
    # 移动端特性（出现频率6.3%）
    "移动特性": [
        "方便", "简便","手机操作", "触屏优化", "云游戏", "便携性",
        "移动存档", "跨平台", "手柄支持", "陀螺仪",
        "移动专属", "离线模式", "省电优化"
    ]
}



    
    # ========== 执行流程 ==========
    # 数据加载与清洗
    raw_df = load_data(FILE_PATH, TEXT_VAR, OTHER_VARS)
    clean_df = clean_text(raw_df, TEXT_VAR)


    # 修改词云生成调用
    generate_wordcloud(
        texts=clean_df[TEXT_VAR],
        stopwords=STOPWORDS,
        save_path=WORDCLOUD_PATH  # 新增保存路径参数
    )

    
    # 新增标签匹配（核心新增功能）
    clean_df[["匹配标签", "匹配关键词"]] = clean_df[TEXT_VAR].apply(
        lambda x: pd.Series(manual_tagging(x, TAG_KEYWORDS, negation_words=NEGATION_WORDS))
    )
    
    # 后续分析流程
    generate_wordcloud(clean_df[TEXT_VAR], STOPWORDS)
    cluster_df, cluster_labels = text_clustering(clean_df[TEXT_VAR])
    clean_df["聚类标签"] = cluster_labels
    export_results(clean_df, cluster_df, OUTPUT_PATH)
    print(f"最终分析完成！！文件已保存至：{OUTPUT_PATH}")

最终分析完成！！文件已保存至：C:\Users\lijinghui03\Desktop\UR工作资料-2025\【116-G79】【25年7月满意度&官网】新界面调研-202507\survey_83426_w_0【合并】[Q37].xlsx
